## Create Python Lambda Layers
Create layers for 
* opensearch
* AWSSigAuth

Example below:

### AWS Lambda Managed Layers
An AWS Professional Service open source python initiative that extends the power of the pandas library to AWS, connecting DataFrames and AWS data & analytics services.

Easy integration with Athena, Glue, Redshift, Timestream, OpenSearch, Neptune, QuickSight, Chime, CloudWatchLogs, DynamoDB, EMR, SecretManager, PostgreSQL, MySQL, SQLServer and S3 (Parquet, CSV, JSON and EXCEL).

Built on top of other open-source projects like Pandas, Apache Arrow and Boto3, it offers abstracted functions to execute your usual ETL tasks like load/unloading data from Data Lakes, Data Warehouses and Databases, even at scale.

**AWS SDK for pandas (awswrangler)**
List of Layers per Region:

https://aws-sdk-pandas.readthedocs.io/en/stable/layers.html

awswrangler [more info](https://pypi.org/project/awswrangler/)




In [1]:
import os
import json
import boto3
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


os.environ['AWS_PROFILE'] = 'isenguard-us-west-2'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2' 
print(os.getenv('AWS_PROFILE'))

vector_store_name = 'image-search-multimodal'
index_name = "image-search-multimodal-index"
encryption_policy_name = "image-search-multimodal-ep"
network_policy_name = "image-search-multimodal-np"
access_policy_name = 'image-search-multimodal-ap'


session = boto3.Session(profile_name='isenguard-us-west-2')
aoss_client = session.client('opensearchserverless')
identity = session.client('sts').get_caller_identity()['Arn']

collections = aoss_client.list_collections(collectionFilters={'name':vector_store_name})
print(collections)
if len(collections) == 0:
    print("No collections found")
    exit(1)
collection_id = collections['collectionSummaries'][0]['id']
print(collection_id)
#host = collection[collection_id] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'
host = collections['collectionSummaries'][0]['id'] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'

host_parts = host.split(':')
region = 'us-west-2'
service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)
client = OpenSearch(
    hosts = [{'host': host_parts[0], 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

bedrock = session.client(
    service_name="bedrock-runtime",
    region_name="us-west-2",
    endpoint_url="https://bedrock-runtime.us-west-2.amazonaws.com"
)


isenguard-us-west-2
{'collectionSummaries': [{'arn': 'arn:aws:aoss:us-west-2:682723651788:collection/1axvs14gqi7wn14ixtm2', 'id': '1axvs14gqi7wn14ixtm2', 'name': 'image-search-multimodal', 'status': 'ACTIVE'}], 'ResponseMetadata': {'RequestId': 'edd4a4d0-de2e-42fb-a8b6-e865b7d76584', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'edd4a4d0-de2e-42fb-a8b6-e865b7d76584', 'date': 'Tue, 11 Jun 2024 02:01:18 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '182', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
1axvs14gqi7wn14ixtm2


In [6]:
# env var
host = '1axvs14gqi7wn14ixtm2.us-west-2.aoss.amazonaws.com'
index = 'image-search-multimodal-index'
region = 'us-west-2'
service = 'aoss'
model = 'amazon.titan-embed-image-v1'
cloudfrontURL = 'https://d9yx5bzoplulh.cloudfront.net/'

# user input from upload form
image_title = 'test'
image_user_labels = 'rekognition'
image_class = 'sealink'

def get_embedding_for_text(text):
    # Bedrock Runtime client used to invoke the model
    bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name=region)
    body = json.dumps(
        {
            "inputText": text
        }
    )
    # The actual call to retrieve embeddings from the model
    response = bedrock_runtime.invoke_model(
        body=body, 
        modelId=model, 
        accept='application/json', 
        contentType='application/json'
    )
    vector_json = json.loads(response['body'].read().decode('utf8'))
    return vector_json, text


def detect_labels(photo, bucket):
    client = session.client('rekognition', region_name=region)
    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
        MaxLabels=10,
        MinConfidence=85,
        # Uncomment to use image properties and filtration settings
        #Features=["GENERAL_LABELS", "IMAGE_PROPERTIES"],
        #Settings={"GeneralLabels": {"LabelInclusionFilters":["Cat"]},
        # "ImageProperties": {"MaxDominantColors":10}}
    )
    print('Detected labels for ' + photo)
    label_names = [label['Name'] for label in response['Labels']]
    return label_names
    
def index_document(document):

    response = wr.opensearch.index_documents(
        client=os_client,
        index=index,
        documents=[document],
    )
    return
label_string = 'bedrock'
text_embedding = get_embedding_for_text(label_string)
print(text_embedding)



({'embedding': [0.008483887, 0.0042419434, -0.016723633, -0.013244629, -0.012756348, 0.024780273, 0.13769531, -0.0022583008, -0.0025024414, 0.024414062, 0.019897461, 0.004852295, 0.029785156, -0.08544922, 0.045410156, 0.01373291, -0.0390625, -0.019897461, 0.052246094, -0.01977539, 0.046875, -0.0078125, 0.010925293, 0.009338379, 0.006591797, -0.044433594, 0.010009766, -0.012573242, 0.0064697266, -0.020874023, 0.016723633, -0.020141602, -0.0044555664, 0.026245117, 0.0044555664, -0.0032653809, 0.030761719, -0.020629883, 0.01965332, 0.018188477, 0.013122559, 0.006378174, -0.061035156, 0.013977051, 0.009887695, 0.026489258, 0.02368164, -0.029418945, 0.05493164, 0.0071105957, 0.0052490234, 0.041503906, -0.011413574, -0.020874023, 0.0036010742, 0.087402344, -0.052490234, -0.03857422, -0.0026550293, -0.035888672, -0.0030059814, -0.012390137, -0.009094238, -0.052001953, 0.005279541, 0.037597656, 0.045166016, 0.076171875, -0.07421875, -0.075683594, -0.013793945, 0.008239746, -0.036621094, -0.031

In [16]:
photo = '002.jpg'

document = {

    "image_path": photo,
    "image_title": image_title,
    "image_labels": label_string,
    "image_class": image_class,
    "image_url": cloudfrontURL + photo, 
    "multimodal_vector": text_embedding[0]['embedding']
}
#print(document)


In [17]:
response = {
    "statusCode": 200,
    "headers": {
        "Access-Control-Allow-Origin": '*'
    },
    "isBase64Encoded": False
}

try:
    #index_document(document)
    index_document = client.index(body=document,index=index)
    print('\nAdding document:')
    print(index_document)
    response['body'] = document
except Exception as ex: 
    print(ex)


Adding document:
{'_index': 'image-search-multimodal-index', '_id': '1%3A0%3AA14XBZABJ7FHc9GUgCn-', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}


In [15]:
embedding_value = text_embedding[0]['embedding']

print(embedding_value)

[0.008483887, 0.0042419434, -0.016723633, -0.013244629, -0.012756348, 0.024780273, 0.13769531, -0.0022583008, -0.0025024414, 0.024414062, 0.019897461, 0.004852295, 0.029785156, -0.08544922, 0.045410156, 0.01373291, -0.0390625, -0.019897461, 0.052246094, -0.01977539, 0.046875, -0.0078125, 0.010925293, 0.009338379, 0.006591797, -0.044433594, 0.010009766, -0.012573242, 0.0064697266, -0.020874023, 0.016723633, -0.020141602, -0.0044555664, 0.026245117, 0.0044555664, -0.0032653809, 0.030761719, -0.020629883, 0.01965332, 0.018188477, 0.013122559, 0.006378174, -0.061035156, 0.013977051, 0.009887695, 0.026489258, 0.02368164, -0.029418945, 0.05493164, 0.0071105957, 0.0052490234, 0.041503906, -0.011413574, -0.020874023, 0.0036010742, 0.087402344, -0.052490234, -0.03857422, -0.0026550293, -0.035888672, -0.0030059814, -0.012390137, -0.009094238, -0.052001953, 0.005279541, 0.037597656, 0.045166016, 0.076171875, -0.07421875, -0.075683594, -0.013793945, 0.008239746, -0.036621094, -0.03125, 0.001853942